# multiobj.py

notebook 版本

source: https://www.gurobi.com/documentation/7.0/examples/multiobj_py.html

这个多目标优化的文档太少

want to cover three different sets but subject to a common budget of
elements allowed to be used. However, the sets have different priorities to
be covered; and we tackle this by using multi-objective optimization.

覆盖 3 种 set，目标是通用的预算控制

Example| 	Description| 	Available Languages
------|----------------|---------------------------
multiobj |	Demonstrates the use of multi-objective optimization.	| C, C++, C#, Java, Python, VB

Other Examples: http://www.gurobi.com/resources/examples/example-models-overview

## Gurobi mutiobj reference 

https://www.gurobi.com/documentation/7.0/refman/multiple_objectives.html

Gurobi 支持两种模式
* In a blended approach, you optimize a weighted combination of the individual objectives.  `SetObjWeight`?
* In a hierarchical or lexicographic approach, you set a priority for each objective, and optimize in priority order.  `SetObjPriority`?

### Step 1: Import functions from the gurobipy module

In [23]:
from __future__ import print_function
from gurobipy import *


# Sample data

# 4 组 suset, 长度 20
Groundset = range(20)
Subsets   = range(4)

# 预算 < 12
Budget    = 12;

Set = [ [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ],
        [ 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1 ],
        [ 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0 ],
        [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0 ] ]
SetObjPriority = [  3,    2,    2,   1]
SetObjWeight   = [1.0, 0.25, 1.25, 1.0]

# Create initial model
model = Model('multiobj')



### Step 2: Create empty model

In [24]:
# Create initial model
model = Model('multiobj')

### Step 3: Create activitiy variables

In [25]:
# Initialize decision variables for ground set:
# x[e] == 1 if element e is chosen for the covering.
Elem = model.addVars(Groundset, vtype=GRB.BINARY, name='El')


### Step 4: Set objective function

Source： https://www.gurobi.com/documentation/7.0/refman/working_with_multiple_obje.html

## Blended Objectives 
`model.ObjNWeight   = SetObjWeight[i]` 权值定义的优化, 相当于混合不同的目标。例如
if your model has two objectives, $1 + x + 2y$ and $y
+ 2z$, and if you give weights of $-1$ and $2$ to them, respectively, then Gurobi would solve your model with a blended objective of $-1 \cdot (1 + x + 2y) + 2 \cdot (y + 2z) = -1 - x + 4z$. 

## Hierarchical Objectives

` model.ObjNPriority = SetObjPriority[i]` 通过设定整数（如 [  3,    2,    2,   1] ） 的优先级来完成

## Combining Blended and Hierarchical Objectives
二者混合

When you specify a different priority for each of $n$ objectives, the solver performs $n$ separate optimization steps. In each step, in decreasing priority order, it optimizes for the current objective, while imposing constraints that ensure that the quality of higher-priority objectives isn't degraded by more than the specified tolerances

One subtle point when blending multiple objectives within a single level in a hierarchical approach relates to the handling of degradations from lower-priority levels. The objective degradation allowed after a blended optimization step is the maximum absolute and relative degradations allowed by each of the participating objectives. For example, 


通过设定不同的优先级， 可以计算最大对最高优先级的最大容忍度

if we have three objectives with ObjNPriority equal to $\{2, 2, 1\}$, and ObjNRelTol equal to  $\{0.10, 0.05, 0.00\}$ and ObjNAbsTol equal to $\{0,
1, 2\}$; and the best solution for the first priority objective is $10$; then the allowed degradation for the first priority objective is $\max\{10 \cdot 0.10, 10 \cdot 0.05, 0, 1\} = 1$.


In [34]:
# Set global sense for ALL objectives
model.ModelSense = GRB.MAXIMIZE

# Limit how many solutions to collect
model.setParam(GRB.Param.PoolSolutions, 100)

# 计划优化的目标数  Objectives are numbered 0 through NumObj-1.
# Set number of objectives 
model.NumObj = 4

# Set and configure i-th objective
# 这里的目标是什么呢？
for i in Subsets:
    model.setParam(GRB.Param.ObjNumber, i)
    # 指定 优先级 和 权值
    model.ObjNPriority = SetObjPriority[i]
    model.ObjNWeight   = SetObjWeight[i]

    model.ObjNName = 'Set' + str(i)
    # 相对容忍度 和绝对容忍度
    model.ObjNRelTol = 0.01
    model.ObjNAbsTol = 1.0 + i
    model.setAttr(GRB.Attr.ObjN, Elem, Set[i])

# Save problem
# model.write('multiobj.lp')


### Step 5: Add constraints

In [27]:
# Constraint: limit total number of elements to be picked to be at most
# Budget
model.addConstr(Elem.sum() <= Budget, name='Budget')

<gurobi.Constr *Awaiting Model Update*>

### Step 6: Solve model

solve logging: https://www.gurobi.com/documentation/7.0/refman/multi_objective_logging.html#sec:MultiObjectiveLogging

In [28]:
# Optimize
model.optimize()

model.setParam(GRB.Param.OutputFlag, 0)



Optimize a model with 1 rows, 20 columns and 20 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 4 objectives (3 combined) ...
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve time: 0.00s
Presolved: 1 rows and 20 columns
---------------------------------------------------------------------------

Multi-objectives: optimize objective 1 (Set0) ...
---------------------------------------------------------------------------

Found heuristic solution: objective 10

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of

In [29]:
# Status checking
status = model.Status
if status == GRB.Status.INF_OR_UNBD or \
   status == GRB.Status.INFEASIBLE  or \
   status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is infeasible or unbounded')
    sys.exit(1)

if status != GRB.Status.OPTIMAL:
    print('Optimization was stopped with status ' + str(status))
    sys.exit(1)

# Print best selected set
print('Selected elements in best solution:')
for e in Groundset:
    if Elem[e].X > 0.9:
        print(' El%d' % e, end='')
print('')

# Print number of solutions stored
nSolutions = model.SolCount
print('Number of solutions found: ' + str(nSolutions))

# Print objective values of solutions
if nSolutions > 10:
    nSolutions = 10
print('Objective values for first ' + str(nSolutions) + ' solutions:')
for i in Subsets:
    model.setParam(GRB.Param.ObjNumber, i)
    print('\tSet%d' % i, end='')
    for e in range(nSolutions):
        model.setParam(GRB.Param.SolutionNumber, e)
        print(' %6g' % model.ObjNVal, end='')
    print('')

Selected elements in best solution:
 El1 El2 El3 El4 El5 El6 El7 El8 El9 El10 El16 El17
Number of solutions found: 4
Objective values for first 4 solutions:
	Set0      9      9     10     10
	Set1      7      7      5      5
	Set2      6      7      6      4
	Set3      7      6      4      4


### Step 8: Print variable values for optimal solution

In [36]:
model.printAttr('X')

In [31]:
model.getVars()

[<gurobi.Var El[0] (value -0.0)>,
 <gurobi.Var El[1] (value 1.0)>,
 <gurobi.Var El[2] (value 1.0)>,
 <gurobi.Var El[3] (value 1.0)>,
 <gurobi.Var El[4] (value 1.0)>,
 <gurobi.Var El[5] (value 1.0)>,
 <gurobi.Var El[6] (value 1.0)>,
 <gurobi.Var El[7] (value 1.0)>,
 <gurobi.Var El[8] (value 1.0)>,
 <gurobi.Var El[9] (value 1.0)>,
 <gurobi.Var El[10] (value 1.0)>,
 <gurobi.Var El[11] (value 0.0)>,
 <gurobi.Var El[12] (value 0.0)>,
 <gurobi.Var El[13] (value 0.0)>,
 <gurobi.Var El[14] (value 0.0)>,
 <gurobi.Var El[15] (value 0.0)>,
 <gurobi.Var El[16] (value 1.0)>,
 <gurobi.Var El[17] (value 1.0)>,
 <gurobi.Var El[18] (value -0.0)>,
 <gurobi.Var El[19] (value 0.0)>]

In [32]:
model.getConstrs()

[<gurobi.Constr Budget>]

In [ ]:
printSolution()

# # 增加不可行条件
